# 健身数据解析

In [1]:
import numpy as np
import pandas as pd
import csv
import os
import struct
from datetime import datetime
from zoneinfo import ZoneInfo
from dataclasses import dataclass, field
from typing import List, Optional,Tuple
from scipy import signal
from scipy.signal import resample
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.dates as mdates


import matplotlib.pyplot as plt
import seaborn as sns
# 设置matplotlib参数
plt.style.use('default')  # 使用默认样式

# 尝试设置中文字体支持(如果失败则使用默认字体)
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
except:
    print("注意: 中文字体设置失败,将使用默认字体")
    pass

from script.data_structures import BaseInfo, ACC_Raw_Data, WearInfo, PPG_Raw_Data, TYHX_PPG_Data
from script.data_types import get_full_data_type
from script.common import (get_raw_data, 
                           parse_filename, 
                           segment_by_labels,
                           print_segments,
                           expand_acc_data,
                           expand_gyro_data, 
                           expand_ppg_data, 
                           process_and_merge_sensor_data)

from script.wear_detection_dll import *
from script.motion_recognition_dll import *

In [2]:
def save_acc_data_as_txt(final_merged_df, file_name_without_ext,dictory):
    expanded_acc_df = expand_acc_data(final_merged_df)
    json_shake_path = os.path.join(dictory, r"data/json_shake")
    data_list_path = os.path.join(dictory, r"data/creek2_step_shake_data_list.txt")
    if not os.path.exists( json_shake_path):
        os.makedirs( json_shake_path)

    acc_file_path = os.path.join(json_shake_path, file_name_without_ext + ".txt")
    data_lines = [f"{int(row['x'])} {int(row['y'])} {int(row['z'])}" 
                for _, row in expanded_acc_df[['x', 'y', 'z']].iterrows()]
    data_str = '\n'.join(data_lines)

    output = f"""header:
    {{
    waist_omron_steps: 0,
    collar_omron_steps: 0,
    reference_steps: 0,
    standard_steps: 0
    }}
    contents:
    {{
    {data_str}
    }}"""

    with open(acc_file_path, 'w') as f:
        f.write(output)



    # 将文件名写入 data/creek2_step_shake_data_list.txt
    with open(data_list_path, 'a') as f:
        f.write(file_name_without_ext + '\n') 


In [3]:
def plot_acc_and_step_data(expanded_acc_df, base_info_df, directory, file_name_without_ext, is_show=False):
    """
    绘制加速度数据和计步数据
    
    参数:
        expanded_acc_df: 展开的加速度数据 DataFrame (包含 x, y, z, unix_timestamp 列)
        base_info_df: 基础信息 DataFrame (包含 unix_timestamp, step 列)
        directory: 保存目录
        file_name_without_ext: 文件名(不含扩展名)
        is_show: 是否显示图表(默认为 False, 保存到文件)
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True)
    
    # 添加文件名作为整个图的标题
    fig.suptitle(file_name_without_ext, fontsize=14, fontweight='bold', y=0.995)
    
    # ax1: 绘制展开的 acc 数据 (x, y, z)
    if not expanded_acc_df.empty and all(col in expanded_acc_df.columns for col in ['x', 'y', 'z']):
        ax1.plot(expanded_acc_df['unix_timestamp'],expanded_acc_df['x'], label='ACC X', color='red', linewidth=0.5)
        ax1.plot(expanded_acc_df['unix_timestamp'],expanded_acc_df['y'], label='ACC Y', color='green', linewidth=0.5)
        ax1.plot(expanded_acc_df['unix_timestamp'],expanded_acc_df['z'], label='ACC Z', color='blue', linewidth=0.5)
        ax1.set_ylabel('Acceleration')
        ax1.set_title(f'Acceleration Data - {len(expanded_acc_df)} samples')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
    
    # ax2: 绘制 step 和 step 的差分
    if not base_info_df.empty and 'step' in base_info_df.columns:
        # 计算步数差分
        step_values = base_info_df['step'].values
        step_diff = np.diff(step_values, prepend=step_values[0])
        
        # 获取起始、结束和总步数
        step_start = base_info_df['step'].iloc[0]
        step_end = base_info_df['step'].iloc[-1]
        step_total = step_end - step_start
        
        # 绘制步数差分(每秒步数)
        # 使用 'o-' 绘制点+线的效果
        ax2.plot(base_info_df['unix_timestamp'], step_diff, '.-', label='Steps/sec', color='purple', 
                 markersize=2, linewidth=0.8, alpha=0.7)
        ax2_twin = ax2.twinx()
        ax2_twin.plot(base_info_df['unix_timestamp'], base_info_df['step'], label='Cumulative Steps', color='orange', linewidth=1.0, alpha=0.7)
        ax2_twin.set_ylabel('Cumulative Steps')
        ax2.set_xlabel('Time (Unix Timestamp)' if 'unix_timestamp' in base_info_df.columns else 'Sample Index')
        ax2.set_ylabel('Steps per Second')
        ax2.set_title(f'Step Data - start:{step_start}, stop:{step_end}, total:{step_total} Sport time:{ base_info_df['unix_timestamp'].max() - base_info_df['unix_timestamp'].min() } sec')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if is_show:
        plt.show()
    else:
        save_fig_name = os.path.join(directory, f"{file_name_without_ext}_acc_step.png")
        plt.savefig(save_fig_name, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"图表已保存: {save_fig_name}")


In [4]:
def plot_acc_and_gyro_data(expanded_acc_df, expanded_gyro_df, directory, file_name_without_ext, is_show=False, segments=None):
    """
    绘制加速度数据和陀螺仪数据
    
    参数:
        expanded_acc_df: 展开的加速度数据 DataFrame (包含 x, y, z, unix_timestamp 列)
        expanded_gyro_df: 展开的陀螺仪数据 DataFrame (包含 x, y, z, unix_timestamp 列)
        directory: 保存目录
        file_name_without_ext: 文件名(不含扩展名)
        is_show: 是否显示图表(默认为 False, 保存到文件)
        segments: 分段信息列表,每个分段包含 start_unix 和 stop_unix
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True)
    
    # 添加文件名作为整个图的标题
    fig.suptitle(file_name_without_ext, fontsize=14, fontweight='bold', y=0.995)
    
    # ax1: 绘制展开的 acc 数据 (x, y, z)
    if not expanded_acc_df.empty and all(col in expanded_acc_df.columns for col in ['x', 'y', 'z']):
        ax1.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['x'], label='ACC X', color='red', linewidth=0.5)
        ax1.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['y'], label='ACC Y', color='green', linewidth=0.5)
        ax1.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['z'], label='ACC Z', color='blue', linewidth=0.5)
        ax1.set_ylabel('Acceleration')
        ax1.set_title(f'Acceleration Data - {len(expanded_acc_df)} samples')
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)
    
    # ax2: 绘制展开的 gyro 数据 (x, y, z)
    if not expanded_gyro_df.empty and all(col in expanded_gyro_df.columns for col in ['x', 'y', 'z']):
        ax2.plot(expanded_gyro_df['unix_timestamp'], expanded_gyro_df['x'], label='GYRO X', color='darkred', linewidth=0.5)
        ax2.plot(expanded_gyro_df['unix_timestamp'], expanded_gyro_df['y'], label='GYRO Y', color='darkgreen', linewidth=0.5)
        ax2.plot(expanded_gyro_df['unix_timestamp'], expanded_gyro_df['z'], label='GYRO Z', color='darkblue', linewidth=0.5)
        ax2.set_xlabel('Time (Unix Timestamp)')
        ax2.set_ylabel('Angular Velocity')
        ax2.set_title(f'Gyroscope Data - {len(expanded_gyro_df)} samples')
        ax2.legend(loc='upper right')
        ax2.grid(True, alpha=0.3)
    
    # 绘制 segments(如果提供)
    if segments:
        for idx, segment in enumerate(segments):
            start_unix = segment.get('start_unix')
            stop_unix = segment.get('stop_unix')
            segment_id = segment.get('segment_id', idx + 1)
            
            if start_unix is not None and stop_unix is not None:
                # 在两个子图上都绘制分段标记
                for ax in [ax1, ax2]:
                    # 绘制起始线(绿色虚线)
                    ax.axvline(x=start_unix, color='green', linestyle='--', linewidth=1.5, alpha=0.7)
                    # 绘制结束线(红色虚线)
                    ax.axvline(x=stop_unix, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
                    # 添加背景色块
                    ax.axvspan(start_unix, stop_unix, alpha=0.1, color='yellow')
                
                # 在 ax1 上添加分段标签
                mid_time = (start_unix + stop_unix) / 2
                ax1.text(mid_time, ax1.get_ylim()[1] * 0.95, f'Segment {segment_id}', 
                        horizontalalignment='center', verticalalignment='top',
                        fontsize=10, fontweight='bold', color='black',
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))
    
    plt.tight_layout()
    
    if is_show:
        plt.show()
    else:
        save_fig_name = os.path.join(directory, f"{file_name_without_ext}_acc_gyro.png")
        plt.savefig(save_fig_name, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"图表已保存: {save_fig_name}")

In [4]:

# directory = r'rawdata/Motion/0108/'

# all_files = os.listdir(directory)

# # 过滤出 .rawdata 文件
# rawdata_files = [f for f in all_files if f.endswith('.rawdata')]
# rawdata_files

In [5]:
# %matplotlib qt 
%matplotlib inline




directory = r'G:\PyScript\rawdata\Fitness_data'
file_name = "HeartRate_Dumbbell_lsj_20251127_180053_7e49.rawdata"



file_path = os.path.join(directory, file_name)
real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
save_dir = os.path.join(directory, "data")
os.makedirs(save_dir, exist_ok=True)
print(f"real_path: {real_path} \n{directory} {file_name_without_ext} {ext}")



result = get_raw_data(real_path)

metadata = result.get('metadata', {})
data_type_dict = result.get('data_types', {})
base_info_df = result.get('base_info_df')
tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))

wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
motion_recognition_df = result.get('motion_recognition_df')

acc_df =  result.get('acc_df', pd.DataFrame(columns=["unix_timestamp","arr_size","acc_data"]))
gyro_df =  result.get('gyro_df', pd.DataFrame(columns=["unix_timestamp","arr_size","gyro_data"]))


# 处理 acc_df - 合并重复的 unix_timestamp
processed_acc_df = acc_df[['unix_timestamp', 'acc_data']].copy()  
if processed_acc_df['unix_timestamp'].duplicated().any():
    print(f"acc_df 发现 {processed_acc_df['unix_timestamp'].duplicated().sum()} 条重复的 unix_timestamp, 正在合并...")
    processed_acc_df = processed_acc_df.groupby('unix_timestamp', as_index=False).agg({
        'acc_data': lambda x: np.concatenate([arr for arr in x if isinstance(arr, np.ndarray)]) if any(isinstance(arr, np.ndarray) for arr in x) else None,
    })
    print(f"acc_df 合并后记录数: {len(processed_acc_df)}")
processed_acc_df['acc_size'] = processed_acc_df['acc_data'].apply(lambda x: len(x) if isinstance(x, np.ndarray) else 0)
processed_acc_df['acc_size'] = processed_acc_df['acc_size'].astype(int)

# 处理 gyro_df - 合并重复的 unix_timestamp
processed_gyro_df = gyro_df[['unix_timestamp', 'gyro_data']].copy()
if processed_gyro_df['unix_timestamp'].duplicated().any():
    print(f"gyro_df 发现 {processed_gyro_df['unix_timestamp'].duplicated().sum()} 条重复的 unix_timestamp, 正在合并...")
    processed_gyro_df = processed_gyro_df.groupby('unix_timestamp', as_index=False).agg({
        'gyro_data': lambda x: np.concatenate([arr for arr in x if isinstance(arr, np.ndarray)]) if any(isinstance(arr, np.ndarray) for arr in x) else None,
    })
    print(f"gyro_df 合并后记录数: {len(processed_gyro_df)}")
processed_gyro_df['gyro_size'] = processed_gyro_df['gyro_data'].apply(lambda x: len(x) if isinstance(x, np.ndarray) else 0)
processed_gyro_df['gyro_size'] = processed_gyro_df['gyro_size'].astype(int)

expanded_acc_df = expand_acc_data(processed_acc_df)
expanded_gyro_df = expand_gyro_data(processed_gyro_df)

expanded_acc_df.to_csv(f"{save_dir}/{file_name_without_ext}_acc_data.csv", index=False, encoding='utf-8')
expanded_gyro_df.to_csv(f"{save_dir}/{file_name_without_ext}_gyro_data.csv", index=False, encoding='utf-8')

real_path: G:\PyScript\rawdata\Fitness_data\HeartRate_Dumbbell_lsj_20251127_180053_7e49.rawdata 
G:\PyScript\rawdata\Fitness_data HeartRate_Dumbbell_lsj_20251127_180053_7e49 .rawdata
acc_df 发现 18343 条重复的 unix_timestamp, 正在合并...
acc_df 合并后记录数: 2374
gyro_df 发现 18344 条重复的 unix_timestamp, 正在合并...
gyro_df 合并后记录数: 2373


In [6]:


# 获取标签并分段
labels = metadata['labels']
print(f"原始标签: {labels}")

# 根据标签分段
segments = segment_by_labels(labels)
print_segments(segments)

原始标签: [{'label': 'Start', 'timestamp': '2025-11-27 18:05:15', 'unix_timestamp': 1764266712}, {'label': 'End', 'timestamp': '2025-11-27 18:05:41', 'unix_timestamp': 1764266739}, {'label': 'End', 'timestamp': '2025-11-27 18:06:18', 'unix_timestamp': 1764266776}, {'label': 'Start', 'timestamp': '2025-11-27 18:06:59', 'unix_timestamp': 1764266817}, {'label': 'End', 'timestamp': '2025-11-27 18:07:22', 'unix_timestamp': 1764266839}, {'label': 'End', 'timestamp': '2025-11-27 18:07:35', 'unix_timestamp': 1764266853}, {'label': 'Start', 'timestamp': '2025-11-27 18:11:05', 'unix_timestamp': 1764267063}, {'label': 'End', 'timestamp': '2025-11-27 18:11:49', 'unix_timestamp': 1764267107}, {'label': 'End', 'timestamp': '2025-11-27 18:11:55', 'unix_timestamp': 1764267113}, {'label': 'Start', 'timestamp': '2025-11-27 18:13:11', 'unix_timestamp': 1764267189}, {'label': 'End', 'timestamp': '2025-11-27 18:13:45', 'unix_timestamp': 1764267223}, {'label': 'Start', 'timestamp': '2025-11-27 18:16:02', 'unix_

In [ ]:
%matplotlib qt 


plot_acc_and_gyro_data(expanded_acc_df, expanded_gyro_df, directory, file_name_without_ext, is_show=True, segments=segments)

In [ ]:




# # 按照 unix_timestamp 合并 acc 和 gyro 数据
# merged_df = pd.merge(processed_acc_df, processed_gyro_df, on='unix_timestamp', how='outer')
# print(f"\n合并后的 DataFrame 记录数: {len(merged_df)}")
# print(f"acc_df 原始记录数: {len(acc_df)}, gyro_df 原始记录数: {len(gyro_df)}")

# merged_df.head()

acc_df 发现 18343 条重复的 unix_timestamp, 正在合并...
acc_df 合并后记录数: 2374
gyro_df 发现 18344 条重复的 unix_timestamp, 正在合并...
gyro_df 合并后记录数: 2373


In [8]:
expanded_acc_df.plot(x='unix_timestamp', y=['x', 'y', 'z'], figsize=(15, 6), title='Expanded ACC Data')

<Axes: title={'center': 'Expanded ACC Data'}, xlabel='unix_timestamp'>

In [ ]:
# %matplotlib qt 

# plot_acc_and_step_data(expanded_acc_df, df, directory, file_name_without_ext, is_show=True)

In [5]:
# expanded_acc_df.plot()

<Axes: >

In [ ]:
# 统计每个步数值出现的次数
df['step'].value_counts().sort_index()

step
4514    45
4524     1
4526     1
4528     1
4529     1
        ..
7421     1
7423     1
7425     1
7427     1
7429    56
Name: count, Length: 1306, dtype: int64

In [25]:
df['step'].value_counts()

step
4971    531
5936    335
6442    290
5473    193
6633    182
       ... 
7382      1
7384      1
7387      1
7389      1
4569      1
Name: count, Length: 1306, dtype: int64

In [10]:

directory = r'G:\python\rawdata_motion\data2'

# G:\python\rawdata_motion\Typing
directory = r'G:\python\rawdata_motion\Typing'



directory = r'rawdata/Motion/251215'


# 获取目录下的所有文件
all_files = os.listdir(directory)

# 过滤出 .rawdata 文件
rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

for f in rawdata_files:
    file_path = os.path.join(directory, f)

    real_path = os.path.realpath(file_path)
    file_name = os.path.basename(file_path)
    file_name_without_ext, ext = os.path.splitext(file_name)
    directory = os.path.dirname(real_path)
    info =  parse_filename(file_name)
    mac = info.get('mac','0000').lower()


    result = get_raw_data(real_path)

    data_type_dict = result.get('data_types', {})
    sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
    readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
    print(f"文件 {file_path} 解析完成,包含数据类型: ")
    print(f"{"\n".join(readable_types)}")

    motion_recognition_df = result.get('motion_recognition_df')
    base_info_df = result.get('base_info_df')
    acc_df = result.get('acc_df')
    if acc_df is None or acc_df.empty:
        print(f"文件 {file_path} 中没有加速度数据,跳过处理。")
        continue
    acc_df = acc_df[["unix_timestamp","arr_size","acc_data"]].copy() # type: ignore
    df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore

    step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
    step_diff2 = df['step'].max() - df['step'].min()
    print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")
    # 调用函数处理和合并 DataFrame
    final_merged_df = process_and_merge_dataframes(df, motion_recognition_df, acc_df)
    Motiondll_with_dataframe(final_merged_df, directory, file_name_without_ext)

文件 rawdata/Motion/251215\HeartRate_IndoorRun_dlh_20251216_170427_5c05.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 614 
15  RAW_PPG_GREEN         原始PPG绿光                count: 642 
17  RAW_PPG_IR            原始PPG红外                count: 642 
18  RAW_ACC               原始加速度                  count: 1284 
21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 1284 
33  AMBIENT_INFO          环境信息                   count: 642 
69  MOTION_INFO           运动识别                   count: 596 
71  WEAR_INFO             佩戴信息                   count: 642 
82  TYHX_AGC_DATA         TYHX AGC数据             count: 642 
step start:7072 stop:8383   diff: 1311  step2: 1311   
日志已保存至: G:\PyScript\rawdata\Motion\251215\HeartRate_IndoorRun_dlh_20251216_170427_5c05_log.txt
文件 G:\PyScript\rawdata\Motion\251215\HeartRate_IndoorRun_dlh_20251216_170427_e61b.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 613 
15  RAW_PPG_GREEN         原始PPG绿光         